In [61]:
import oracledb, os, time

In [62]:
# running oracle-free image

def create_connection():
    try:
        connection = oracledb.connect(
            user="SYSTEM",
            password=os.getenv("ORACLE_PASSWORD"),
            dsn="localhost:1523/FREE"          
        )
        print("Verbindung zur PDB erfolgreich!")
    except oracledb.DatabaseError as e:
        print(f"Verbindung fehlgeschlagen: {e}")
    return connection

# Queries

In [63]:
#-- 1) Durchschnittliche Luftfeuchtigkeit (humidity) pro Gerät
average_humidity_per_device_query = """SELECT
  DEVICE,
  AVG(HUMIDITY) AS avg_humidity
FROM IOT_TELEMETRY_DATA
GROUP BY DEVICE
ORDER BY DEVICE
"""

In [64]:
# -- 2) Höchste gemessene Temperatur über alle Geräte
highest_temp_query = """SELECT
  MAX(temp) AS max_temperature
FROM IOT_TELEMETRY_DATA
"""

In [65]:
# -- 3) “Worst air quality” per Gerät als Summe der drei Schadstoffe
worst_air_quality_per_device_query = """SELECT
  device,
  SUM(co + lpg + smoke) AS air_quality_score
FROM IOT_TELEMETRY_DATA
GROUP BY device
ORDER BY air_quality_score DESC
"""

In [66]:
# Query: Tages-AVG pro Gerät + Abweichung in %
diff_query = """
SELECT
  ts,
  device,
  temp,
  AVG(temp) OVER (PARTITION BY device) AS avg_temp_device,
  ROUND(
    ABS(
      temp 
      - AVG(temp) OVER (PARTITION BY device)
    )
    / AVG(temp) OVER (PARTITION BY device)
    * 100
  , 2) AS per_diff_pct
FROM iot_telemetry_data
ORDER BY device, ts
"""

In [67]:
hourly_temp_hum_query = """
SELECT
    TRUNC(TS, 'HH') AS HOUR_WINDOW, -- Zeitstempel-Spalte 'TS' auf Stunde runden
    DEVICE, -- Geräte-Spalte 'DEVICE'
    AVG(TEMP) AS MEAN_TEMP, -- Durchschnitt der Temperatur-Spalte 'TEMP'
    MAX(HUMIDITY) AS MAX_HUMIDITY -- Maximum der Luftfeuchtigkeit-Spalte 'HUMIDITY'
FROM
    IOT_TELEMETRY_DATA -- Dein Tabellenname
WHERE
    TS >= TO_TIMESTAMP('2002-04-07 00:00:00', 'YYYY-MM-DD HH24:MI:SS') -- Startzeit (anpassen!)
    AND TS < CURRENT_TIMESTAMP -- Endzeit (jetzt)
    AND DEVICE IN ('00:0f:00:70:91:0a', '1c:bf:ce:15:ec:4d', 'eb:b8:27:bf:9d:51') -- Geräte-IDs
GROUP BY
    TRUNC(TS, 'HH'),
    DEVICE
ORDER BY
    HOUR_WINDOW,
    DEVICE
"""

In [68]:
temperature_rate_of_change_per_minute_by_device = """
WITH DeviceRankedData AS (
    SELECT
        TS,
        DEVICE,
        TEMP,
        LAG(TS, 1) OVER (PARTITION BY DEVICE ORDER BY TS) AS PREV_TS,
        LAG(TEMP, 1) OVER (PARTITION BY DEVICE ORDER BY TS) AS PREV_TEMP
    FROM
        IOT_TELEMETRY_DATA -- Dein Tabellenname
    WHERE
        TS >= TO_TIMESTAMP('2020-07-12 00:00:00', 'YYYY-MM-DD HH24:MI:SS') -- Startzeit deines Bereichs
        AND TS <= TO_TIMESTAMP('2020-07-19 23:59:59', 'YYYY-MM-DD HH24:MI:SS') -- Endzeit deines Bereichs
)
SELECT
    TS,
    DEVICE,
    TEMP,
    CASE
        WHEN PREV_TS IS NULL OR TS = PREV_TS THEN NULL -- Keine Berechnung möglich, wenn kein vorheriger Punkt oder Zeitstempel identisch
        ELSE
            (TEMP - PREV_TEMP) / -- Temperaturdifferenz
            ( -- Zeitdifferenz als Interval, dann in Minuten umrechnen
              EXTRACT(DAY    FROM (TS - PREV_TS)) * 24 * 60 +
              EXTRACT(HOUR   FROM (TS - PREV_TS)) * 60 +
              EXTRACT(MINUTE FROM (TS - PREV_TS)) +
              EXTRACT(SECOND FROM (TS - PREV_TS)) / 60
            )
    END AS TEMP_RATE_OF_CHANGE_PER_MINUTE
FROM
    DeviceRankedData
WHERE
    PREV_TS IS NOT NULL -- Schliesst den ersten Punkt pro Gerät aus
ORDER BY
    DEVICE, TS
"""

In [69]:
time_stamps_query = """
SELECT TS FROM IOT_TELEMETRY_DATA
"""

In [70]:
avg_temp_humidity_ratio_query = """
SELECT
    AVG(TEMP / NULLIF(HUMIDITY, 0)) AS avg_temp_humidity_ratio
FROM
    IOT_TELEMETRY_DATA
WHERE
    TS >= TO_TIMESTAMP('2020-07-14 00:01:00', 'YYYY-MM-DD HH24:MI:SS')
    AND TS <= TO_TIMESTAMP('2020-07-19 03:01:00', 'YYYY-MM-DD HH24:MI:SS')
"""

In [71]:
average_humidity_per_device_query_complex = """
SELECT device, avg_humidity FROM (
    SELECT
        device,
        AVG(humidity) OVER (PARTITION BY device) AS avg_humidity,
        ROW_NUMBER() OVER (PARTITION BY device ORDER BY device) AS rn
    FROM iot_telemetry_data
)
WHERE rn = 1
ORDER BY device
"""

In [72]:
highest_temp_query_complex = """
SELECT max_temp FROM (
    SELECT temp, MAX(temp) OVER () AS max_temp
    FROM iot_telemetry_data
) WHERE ROWNUM = 1
"""

In [73]:
import pandas as pd
try:
    connection = create_connection()
    cursor = connection.cursor()
    cursor.execute(time_stamps_query)
    time_stamps = pd.DataFrame(cursor.fetchall())
    print(time_stamps.min())
    print(time_stamps.max())
    cursor.close()
    connection.close()
except oracledb.DatabaseError as e:
    print("Error reading data:", e)

Verbindung zur PDB erfolgreich!
0   2020-07-12 00:01:34
dtype: datetime64[ns]
0   2020-07-20 00:03:37
dtype: datetime64[ns]


# Performance Analysis

In [74]:
query_times = {}

In [75]:
try:
    connection = create_connection()
    cursor = connection.cursor()
    start_time = time.time()
    cursor.execute(average_humidity_per_device_query)
    avg_humidity = cursor.fetchall()
    end_time = time.time()
    print(f"Successfull fetched in {end_time - start_time} seconds.")
    print(avg_humidity)
    query_times['Average Humidity'] = (end_time - start_time)
    cursor.close()
    connection.close()
except oracledb.DatabaseError as e:
    print("Error reading data:", e)

Verbindung zur PDB erfolgreich!
Successfull fetched in 0.07648849487304688 seconds.
[('00:0f:00:70:91:0a', 75.44436079832734), ('1c:bf:ce:15:ec:4d', 61.91024663855242), ('b8:27:eb:bf:9d:51', 50.814076745389464)]


In [76]:
try:
    connection = create_connection()
    cursor = connection.cursor()
    start_time = time.time()
    cursor.execute(highest_temp_query)
    highest_temp = cursor.fetchall()
    end_time = time.time()
    print(f"Successfull fetched in {end_time - start_time} seconds.")
    print(highest_temp)
    query_times['Highest Temp'] = (end_time - start_time)
    cursor.close()
    connection.close()
except oracledb.DatabaseError as e:
    print("Error: ", e)


Verbindung zur PDB erfolgreich!
Successfull fetched in 0.029315471649169922 seconds.
[(30.600000381469727,)]


In [77]:
try:
    connection = create_connection()
    cursor = connection.cursor()
    start_time = time.time()
    cursor.execute(worst_air_quality_per_device_query)
    worst_air = cursor.fetchall()
    end_time = time.time()
    print(f"Successfull fetched in {end_time - start_time} seconds.")
    print(worst_air)
    query_times['Worst Air'] = (end_time - start_time)
    cursor.close()
    connection.close()
except oracledb.DatabaseError as e:
    print("Error: ", e)

Verbindung zur PDB erfolgreich!
Successfull fetched in 0.07038450241088867 seconds.
[('b8:27:eb:bf:9d:51', 6777.162111500038), ('1c:bf:ce:15:ec:4d', 3054.864279412305), ('00:0f:00:70:91:0a', 2785.2340317640083)]


In [78]:

try:
    connection = create_connection()
    cursor = connection.cursor()
    start_time = time.time()
    cursor.execute(avg_temp_humidity_ratio_query)
    worst_air = cursor.fetchall()
    end_time = time.time()
    print(f"Successfull fetched in {end_time - start_time} seconds.")
    print(worst_air)
    query_times['Avg Temp Humidity Ratio'] = (end_time - start_time)
    cursor.close()
    connection.close()
except oracledb.DatabaseError as e:
    print("Error: ", e)

Verbindung zur PDB erfolgreich!
Successfull fetched in 0.09484338760375977 seconds.
[(0.39024171766859717,)]


# Summary

In [79]:
performance = "OracleDB query times: " + str(query_times)
print(performance)

OracleDB query times: {'Average Humidity': 0.07648849487304688, 'Highest Temp': 0.029315471649169922, 'Worst Air': 0.07038450241088867, 'Avg Temp Humidity Ratio': 0.09484338760375977}


In [80]:
filename = "performance.txt"

with open(filename, "a", encoding="utf-8") as f:
    f.write(performance + "\n")

print(performance)


OracleDB query times: {'Average Humidity': 0.07648849487304688, 'Highest Temp': 0.029315471649169922, 'Worst Air': 0.07038450241088867, 'Avg Temp Humidity Ratio': 0.09484338760375977}
